## Évaluation et sélection du modèle

Cette section vise à comparer les différents modèles de classification entraînés
dans ce projet afin de sélectionner le modèle final le plus adapté au problème
de prédiction du revenu.

In [11]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import random

SEED = 42

np.random.seed(SEED)
random.seed(SEED)

In [3]:
df = pd.read_csv("../data/processed/adult_cleaned.csv")

X = df.drop("income", axis=1)
y = df["income"].map({"<=50K": 0, ">50K": 1})

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [5]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

X_train, X_test = X_train.align(
    X_test,
    join="left",
    axis=1,
    fill_value=0
)

In [6]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_scaled, y_train)

y_pred_lr = log_reg.predict(X_test_scaled)
y_proba_lr = log_reg.predict_proba(X_test_scaled)[:, 1]

In [8]:
rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42
)

rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
y_proba_rf = rf.predict_proba(X_test)[:, 1]

In [9]:
gb = GradientBoostingClassifier(random_state=42)

gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)
y_proba_gb = gb.predict_proba(X_test)[:, 1]

In [10]:
results = pd.DataFrame({
    "Modèle": ["Régression logistique", "Random Forest", "Gradient Boosting"],
    "Accuracy": [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_rf),
        accuracy_score(y_test, y_pred_gb)
    ],
    "F1-score (classe 1)": [
        f1_score(y_test, y_pred_lr),
        f1_score(y_test, y_pred_rf),
        f1_score(y_test, y_pred_gb)
    ],
    "ROC-AUC": [
        roc_auc_score(y_test, y_proba_lr),
        roc_auc_score(y_test, y_proba_rf),
        roc_auc_score(y_test, y_proba_gb)
    ]
})

results

,Modèle,Accuracy,F1-score (classe 1),ROC-AUC
0,Régression logistique,0.852602,0.663630,0.902574
1,Random Forest,0.851528,0.665050,0.901482
2,Gradient Boosting,0.862122,0.676978,0.915406


## Évaluation et sélection du modèle final

Les trois modèles ont été évalués sur le même jeu de test à l’aide de métriques
adaptées à un problème de classification binaire avec déséquilibre des classes :
accuracy, F1-score pour la classe minoritaire et ROC-AUC.

La régression logistique sert de modèle de référence et montre des performances
correctes mais limitées pour la détection de la classe minoritaire.
Le Random Forest améliore ces résultats en capturant des relations non linéaires,
au prix d’une complexité plus élevée.

Le Gradient Boosting obtient les meilleures performances globales, notamment en
termes de F1-score et de ROC-AUC, tout en conservant une bonne capacité de
généralisation. Il est donc retenu comme modèle final pour ce problème.

## Évaluation et sélection du modèle final

Les trois modèles ont été évalués sur le même jeu de test à l’aide de métriques
adaptées à un problème de classification binaire avec déséquilibre des classes :
accuracy, F1-score pour la classe minoritaire et ROC-AUC.

La régression logistique sert de modèle de référence et montre des performances
correctes mais limitées pour la détection de la classe minoritaire.
Le Random Forest améliore ces résultats en capturant des relations non linéaires,
au prix d’une complexité plus élevée.

Le Gradient Boosting obtient les meilleures performances globales, notamment en
termes de F1-score et de ROC-AUC, tout en conservant une bonne capacité de
généralisation. Il est donc retenu comme modèle final pour ce problème.